In [ ]:
!pip install bleak
!pip install yalexs-ble
!pip install bleak-retry-connector

In [7]:
lock_cmd = "LOCK"

In [ ]:
import asyncio
import logging
import json

from bleak import BleakScanner, BLEDevice

from yalexs_ble import Lock, LockState, PushLock, serial_to_local_name
from yalexs_ble.const import ConnectionInfo, LockInfo

from bleak_retry_connector import (
    MAX_CONNECT_ATTEMPTS,
    BleakClientWithServiceCache,
    BLEDevice,
    establish_connection,
)

_LOGGER = logging.getLogger(__name__)

_LOGGER.setLevel(logging.DEBUG)

with open("door_lock.json", "r") as fpdl:
    lock_data = json.load(fpdl)

mac = lock_data["macAddress"]
LOCK_SERIAL = lock_data["serialNumber"]
LOCK_KEY = lock_data["secretKey"]
LOCK_KEY_INDEX = lock_data["slotNumber"]

import asyncio
from bleak import BleakScanner
from yalexs_ble.lock import Lock
from yalexs_ble.lock import LockStateValue

async def find_device(mac_addr: str = None):
    device = await BleakScanner.find_device_by_address(mac_addr, timeout=30)
    return device

def ble_device_callback_factory(device):
    # Returns a callback function that returns this BLE device
    def _callback():
        return device
    return _callback

def state_callback(states):
    # states is an iterable of LockStateValue
    print("Lock states updated:")
    for st in states:
        print("  ", st)

def disconnect_callback():
    print("Lock disconnected!")

async def main():
    # 1. Discover a nearby BLE device (your Yale lock)
    device = await find_device(mac_addr=mac)
    if device is None:
        print("No Yale BLE device found. Exiting.")
        return

    # 2. Create the Lock object
    lock = Lock(
        ble_device_callback=ble_device_callback_factory(device),
        keyString=LOCK_KEY,   # you need the actual key
        keyIndex=LOCK_KEY_INDEX,                         # depends on your key slot
        name=LOCK_SERIAL,
        state_callback=state_callback,
        disconnect_callback=disconnect_callback,
        info=None  # or a LockInfo if you have it
    )

    # 3. Connect / start lock operation
    # The library may have a method to "start" or "connect" - check what Lock supports
    await lock.connect()
    # NOTE: You have to be quick between connect and lock. Don't wait, it will disconnect

    if lock_cmd == "LOCK":
        await lock.lock()

    if lock_cmd == "UNLOCK":
        await lock.unlock()

    # 6. Disconnect / clean up
    await lock.disconnect()

#if __name__ == "__main__":
await main()
